In [1]:
import Parser
import Process_Data
import Feature_Engineering as fe

import datetime
import pandas as pd
import os

## DOWNLOAD DATA

In [ ]:
## FOR MAC 
## Chrome version 90.0.4430.212
script_dir = os.path.dirname(os.path.realpath('chromedriver_mac'))
webdriver_path = os.path.join(script_dir, 'chromedriver_mac')

In [ ]:
## FOR WINDOWS
## Chrome version 91.0.4472.77
# script_dir = os.path.dirname(os.path.realpath('chromedriver.exe'))
# webdriver_path = os.path.join(script_dir, 'chromedriver.exe')
##

In [20]:
## TEST EGOR
## 2021-05-25 - 2021-05-27

In [3]:
## PARSE
matches = []
hrefs = set()

startdate = datetime.date(2021, 5, 24)
enddate = datetime.date(2021, 5, 25)
Parser.parse(startdate, enddate, matches, hrefs, webdriver_path)


## UPDATE OLD MATCHES
filename = 'Data/old_matches.csv'
old_matches = Process_Data.csv_to_list(filename)

matches = Process_Data.unpack_and_change(matches)
Process_Data.make_csv(matches, 'Data/new_matches.csv')
matches_after_parse = Process_Data.csv_to_list('Data/new_matches.csv')
Process_Data.make_csv(old_matches + matches_after_parse, 'Data/old_matches.csv')


## PROCESS OLD MATCHES
Process_Data.delete_trash(old_matches)
Process_Data.split_sets_and_dur(old_matches)
Process_Data.fix_set_score(old_matches)
Process_Data.split_stats(old_matches)
Process_Data.create_srv_games(old_matches)
Process_Data.create_bp_stat(old_matches)


## PROCESS NEW MATCHES
filename = 'Data/new_matches.csv'
new_matches = Process_Data.csv_to_list(filename)
Process_Data.delete_trash(new_matches)
Process_Data.split_sets_and_dur(new_matches)
Process_Data.fix_set_score(new_matches)
Process_Data.split_stats(new_matches)
Process_Data.create_srv_games(new_matches)
Process_Data.create_bp_stat(new_matches)

## UNION MATCHES --> CSV
matches = old_matches + new_matches
filename = 'Data/matches_final.csv'
Process_Data.to_csv(matches, filename)

## CSV --> PICKLE
data = pd.read_csv(filename)
filename = 'Data/matches_final.pkl'
data.to_pickle(filename)


/Users/yasha_ten/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
## THINK OVER
import zipfile
with zipfile.ZipFile("Data/data.zip", "r") as z:
    z.extractall("")

## FEATURE ENGINEERING

In [10]:
## READ DATA
df = pd.read_pickle('Data/matches_final.pkl') 
players1 = pd.read_pickle('Data/players.pkl')
players2 = pd.read_pickle('Data/players.pkl')

## CHANGE TIME TYPE
fe.fix_time(df)

## fix players' names for transporting stats
fe.fix_names1(players1)
fe.fix_names2(players2)
players1 = players1.set_index(['name'])
players2 = players2.set_index(['name'])

## transporting stats players --> dataframe
fe.add_stats(df, players1)
fe.add_stats(df, players2, 0)


## fix letters in data
fe.fix_letters(df)

## create retirement stat
fe.create_retirement_stat(df)


## converting age to datetime
fe.fix_age(df, 1)
fe.fix_age(df, 2)


## converting some stats to datetime in df for the next steps
df['date'] = pd.to_datetime(df['date'])
df['p1_birth'] = pd.to_datetime(df['p1_birth'])
df['p2_birth'] = pd.to_datetime(df['p2_birth'])

# count prc of winning points on return
fe.create_feature_WRP(df)

## normalize to per_game stats
fe.create_aces_per_game(df)
fe.create_double_faults_per_game(df)


## create stats based on common opponents
stats = ['first_serve_prc_match_player1', 'first_serve_points_prc_match_player1',
         'second_serve_points_prc_match_player1', 'winning_on_return_prc_player1',
        'aces_per_game_player1', 'df_per_game_player1', 'break_points_prc_match_player1']
fe.create_common_stats(df, stats)


## create new features
fe.create_feature_WSP(df)
fe.create_feature_COMPLETE(df)
fe.create_feature_SERVEADV(df)

# add invert matches
startdate = '2021-01-01'
df = fe.invert(df, startdate)

## drop Nan stats
df.dropna(subset=['p1_height'], inplace=True)
df.dropna(subset=['p2_height'], inplace=True)
df.dropna(subset=['p1_birth'], inplace=True)
df.dropna(subset=['p2_birth'], inplace=True)
df.dropna(subset=['k1'], inplace=True)
df.dropna(subset=['k2'], inplace=True)

# FIX SOME STATS' FORMAT
df = fe.fix_format(df)

# CREATE LAG FEATURES USING MAIN STATS
df = fe.build_lag_features(df)


df.rename(columns={"player1": "p1", "player2": "p2", "player1_win": "p1_win"}, inplace=True)
df = df.set_index(["date", "p1", "p2"])

df.head(3)

FIXING PLAYERS' NAMES...


IntProgress(value=0, max=1357)

IntProgress(value=0, max=1357)

PLAYERS' NAMES FIXED

IMPORTING STATIC STATS...


IntProgress(value=0, max=200)

...

IntProgress(value=0, max=173)

STAT break_points_prc_match_common CREATED
ADDING INVERT MATCHES...
INVERT MATCHES ADDED

CREATING LAG FEATURES...


/Users/yasha_ten/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


LAG FEATURES CREATED



index  \
date       p1         p2                    
2021-05-03 Alcaraz C. Mannarino A.      0   
           Chardy J.  Evans D.          1   
           Evans D.   Chardy J.         2   

                                                                            match_url  \
date       p1         p2                                                                
2021-05-03 Alcaraz C. Mannarino A.  https://www.sofascore.com/alcaraz-mannarino/Ur...   
           Chardy J.  Evans D.         https://www.sofascore.com/evans-chardy/EqgsACg   
           Evans D.   Chardy J.        https://www.sofascore.com/evans-chardy/EqgsACg   

                                   status  Surface    k1    k2  p1_win  round  \
date       p1         p2                                                        
2021-05-03 Alcaraz C. Mannarino A.  Ended      0.0  1.20  4.50       1    1.0   
           Chardy J.  Evans D.      Ended      0.0  2.20  1.67       0    1.0   
           Evans D.   Chardy J.     Ended      0.0  1.67  2.20       1    1.0   

                                    match_dur Location  sets_stat  \
date       p1         p2                                            
2021-05-03 Alcaraz C. Mannarino A.       71.0  Madrid         NaN   
           Chardy J.  Evans D.          165.0  Madrid         NaN   
           Evans D.   Chardy J.         165.0  Madrid         NaN   

                                    duration_set1  duration_set2  \
date       p1         p2                                           
2021-05-03 Alcaraz C. Mannarino A.           43.0           28.0   
           Chardy J.  Evans D.               63.0           67.0   
           Evans D.   Chardy J.              63.0           67.0   

                                    duration_set3  duration_set4  \
date       p1         p2                                           
2021-05-03 Alcaraz C. Mannarino A.            NaN            NaN   
           Chardy J.  Evans D.               35.0            NaN   
           Evans D.   Chardy J.              35.0            NaN   

                                    duration_set5  aces_match_player1  \
date       p1         p2                                                
2021-05-03 Alcaraz C. Mannarino A.            NaN                 1.0   
           Chardy J.  Evans D.                NaN                 4.0   
           Evans D.   Chardy J.               NaN                12.0   

                                    aces_match_player2  \
date       p1         p2                                 
2021-05-03 Alcaraz C. Mannarino A.                 1.0   
           Chardy J.  Evans D.                    12.0   
           Evans D.   Chardy J.                    4.0   

                                    double_faults_match_player1  \
date       p1         p2                                          
2021-05-03 Alcaraz C. Mannarino A.                          1.0   
           Chardy J.  Evans D.                              6.0   
           Evans D.   Chardy J.                             1.0   

                                    double_faults_match_player2  \
date       p1         p2                                          
2021-05-03 Alcaraz C. Mannarino A.                          1.0   
           Chardy J.  Evans D.                              1.0   
           Evans D.   Chardy J.                             6.0   

                                    first_serve_prc_match_player1  \
date       p1         p2                                            
2021-05-03 Alcaraz C. Mannarino A.                       0.642857   
           Chardy J.  Evans D.                           0.677419   
           Evans D.   Chardy J.                          0.754545   

                                    first_serve_prc_match_player2  \
date       p1         p2                                            
2021-05-03 Alcaraz C. Mannarino A.                       0.568966   
           Chardy J.  Evans D.                  

## CREATE MODELS

In [11]:
import Create_Models as cm

In [12]:
# df = pd.read_pickle('test.pkl') ## для теста
predictions = []
# startdate = '2021-05-20'

data = cm.create_data1(df)
y_pred1 = cm.build_model_classifier(data, startdate=startdate)
predictions.append(y_pred1)
data = cm.create_data2(df)
y_pred2 = cm.build_model_classifier(data, startdate=startdate)
predictions.append(y_pred2)
data = cm.create_data3(df)
y_pred3 = cm.build_model_classifier(data, startdate=startdate)
predictions.append(y_pred3)


## BUILD COMPOSITION

In [13]:
import Composition
agg = Composition.aggregate(Composition.get_res(df, startdate), Composition.make_df(predictions))

In [16]:
import zipfile
newzip = zipfile.ZipFile('Data/data.zip','w', zipfile.ZIP_DEFLATED)
newzip.write('Data/all_features.pkl')
newzip.write('Data/old_matches.csv')
newzip.write('Data/players.pkl')
newzip.close() #закрываем архив

In [15]:
with zipfile.ZipFile("Data/data.zip", "r") as z:
    z.extractall("Data/")

In [11]:
## IAN: 2021-01-01 - 2021-03-01 // EGOR: 2021-03-01 - 2021-05-21